# Conjunto de dados

aXXX

# Processamento dos dados

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [ ]:
pip install dalex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.7.2-py3-none-any.whl size=1042898 sha256=03b31f1810c95abc4359647508d3d0f7c1e2d878dbc13aefd06fe2e3de27cbed
  Stored in directory: /root/.cache/pip/wheels/cd/5e/ec/848120d5b913158656c0f8ef316233dd3356effa64aef19fba
Successfully built dalex


In [ ]:
!pip install aif360 numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px



from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn import metrics

from io import StringIO
import sys

import re
import sys
from io import StringIO

import dalex as dx


In [ ]:
df_application = pd.read_csv('application_record.csv')
df_credit = pd.read_csv('credit_record.csv')

In [ ]:
#df = pd.merge(df_application, df_credit, on='ID', how='inner')

In [ ]:
#df.isnull().sum()

In [ ]:
#df.dropna(inplace=True)

In [ ]:
#df_amostra = df.sample(frac=0.70, random_state=42)  # Usa 10% dos dados
#df_amostra.shape

In [ ]:
#df = df_amostra

In [ ]:
#df["CODE_GENDER"].value_counts()

In [ ]:
#df=df.drop(["FLAG_PHONE"], axis=1)

In [ ]:
mode_status = df_credit.groupby('ID')['STATUS'].agg(lambda x: x.mode()[0]).reset_index()
mode_status.rename(columns={'STATUS': 'mode_status'}, inplace=True)
mode_status = mode_status[mode_status['mode_status'] != 'X']

In [ ]:
# Merge the filtered mode_status with application_df
merged_df = pd.merge(df_application, mode_status, on='ID', how='inner')

In [ ]:
# Define the label_status function to categorize the mode_status
def label_status(status):
    if status in {"0", "1", "2", "3", "4", "5"}:
        return 'bad'
    elif status == 'C':
        return 'good'
    else:
        return 'unknown'  # Just in case there's an unexpected status value

In [ ]:
merged_df['label'] = merged_df['mode_status'].apply(label_status)

merged_df['label'].value_counts()

,count
label,
bad,17652
good,12807


In [ ]:
merged_df['TARGET'] = merged_df['label'].apply(lambda x: 0 if x == 'bad' else 1)

merged_df['TARGET'].value_counts()

,count
TARGET,
0,17652
1,12807


In [ ]:
merged_df.drop(columns=['mode_status', 'label'], inplace=True)

In [ ]:
#drop un nessesary columns
merged_df.fillna({'OCCUPATION_TYPE': 'Unknown'}, inplace=True)

In [ ]:
occupation_types = merged_df['OCCUPATION_TYPE'].unique()

In [ ]:
# Compute age from DAYS_BIRTH
merged_df['AGE'] = merged_df['DAYS_BIRTH'].apply(lambda x: abs(x) // 365)

# Calculate credit history length
credit_history_length = df_credit.groupby('ID')['MONTHS_BALANCE'].apply(lambda x: x.max() - x.min()).reset_index()
credit_history_length.columns = ['ID', 'CREDIT_HISTORY_LENGTH']
merged_df = pd.merge(merged_df, credit_history_length, on='ID', how='left')

# Determine if there's recent activity within the last 6 months
def recent_activity(months_balance):
    return 1 if any(month >= -6 for month in months_balance) else 0

recent_activity_flag = df_credit.groupby('ID')['MONTHS_BALANCE'].apply(recent_activity).reset_index()
recent_activity_flag.columns = ['ID', 'RECENT_ACTIVITY']
merged_df = pd.merge(merged_df, recent_activity_flag, on='ID', how='left')

# Calculate years employed and unemployment status
merged_df['YEARS_EMPLOYED'] = merged_df['DAYS_EMPLOYED'].apply(lambda x: abs(x) // 365)
merged_df['IS_UNEMPLOYED'] = merged_df['DAYS_EMPLOYED'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
merged_df['CODE_GENDER'] = le.fit_transform(merged_df['CODE_GENDER'])
merged_df['FLAG_OWN_CAR'] = le.fit_transform(merged_df['FLAG_OWN_CAR'])
merged_df['FLAG_OWN_REALTY'] = le.fit_transform(merged_df['FLAG_OWN_REALTY'])

# Convert categorical features to dummy/indicator variables
merged_df = pd.get_dummies(merged_df,
                           columns=['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                                    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE'],
                           drop_first=True)

# Identify the boolean columns and cast them to integers
bool_columns = merged_df.select_dtypes(include=['bool']).columns
merged_df[bool_columns] = merged_df[bool_columns].astype(int)

In [ ]:
merged_df.drop(columns=['DAYS_BIRTH', 'DAYS_EMPLOYED'], inplace=True)

In [ ]:
merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)

Devido a este conjunto de dados não ter dados rotulados, então rotularei os dados especialmente. Usarei a coluna de status para fazer meu rótulo, pois todos os usuários que atrasam por mais de 60 meses, rotularei-os com SIM (Pessoa Arriscada) e os outros com NÃO.

In [ ]:
Categorical=merged_df.select_dtypes(include='object').columns.tolist()
Categorical

[]

In [ ]:
merged_df.CODE_GENDER.value_counts()

,count
CODE_GENDER,
0,20495
1,9964


In [ ]:
Numerical=merged_df.select_dtypes(exclude='object').columns.tolist()
Numerical

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'TARGET',
 'AGE',
 'CREDIT_HISTORY_LENGTH',
 'RECENT_ACTIVITY',
 'YEARS_EMPLOYED',
 'IS_UNEMPLOYED',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_State servant',
 'NAME_INCOME_TYPE_Student',
 'NAME_INCOME_TYPE_Working',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Incomplete higher',
 'NAME_EDUCATION_TYPE_Lower secondary',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'NAME_FAMILY_STATUS_Married',
 'NAME_FAMILY_STATUS_Separated',
 'NAME_FAMILY_STATUS_Single / not married',
 'NAME_FAMILY_STATUS_Widow',
 'NAME_HOUSING_TYPE_House / apartment',
 'NAME_HOUSING_TYPE_Municipal apartment',
 'NAME_HOUSING_TYPE_Office apartment',
 'NAME_HOUSING_TYPE_Rented apartment',
 'NAME_HOUSING_TYPE_With parents',
 'OCCUPATION_TYPE_Cleaning staff',
 'OCCUPATION_TYPE_Cooking staff',
 'OCCUPATION_

In [ ]:
df = merged_df

In [ ]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unknown,OCCUPATION_TYPE_Waiters/barmen staff
0,5008804,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5008805,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5008810,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
3,5008811,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
4,5008812,0,0,1,0,283500.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df.CODE_GENDER.value_counts()

,count
CODE_GENDER,
0,20495
1,9964


In [ ]:
df.shape

(30459, 52)

# Primeira etapa do experimento

* dataset do compas
* vários algoritmos
* várias métricas de **fairness**

Fairness by unawareness ou justiça através do desconhecimento é um conceito ingênuo de  justiça em que um algoritmo justo é definido quando não usa nenhum atributo protegido no processo de treinamento. Então não existe essa ideia de treinar sem o atributo protegido.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET",'ID'], axis =1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
categorical_features = []
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = [

 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',

 'AGE',
 'CREDIT_HISTORY_LENGTH',
 'RECENT_ACTIVITY',
 'YEARS_EMPLOYED',
 'IS_UNEMPLOYED',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_State servant',
 'NAME_INCOME_TYPE_Student',
 'NAME_INCOME_TYPE_Working',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Incomplete higher',
 'NAME_EDUCATION_TYPE_Lower secondary',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'NAME_FAMILY_STATUS_Married',
 'NAME_FAMILY_STATUS_Separated',
 'NAME_FAMILY_STATUS_Single / not married',
 'NAME_FAMILY_STATUS_Widow',
 'NAME_HOUSING_TYPE_House / apartment',
 'NAME_HOUSING_TYPE_Municipal apartment',
 'NAME_HOUSING_TYPE_Office apartment',
 'NAME_HOUSING_TYPE_Rented apartment',
 'NAME_HOUSING_TYPE_With parents',
 'OCCUPATION_TYPE_Cleaning staff',
 'OCCUPATION_TYPE_Cooking staff',
 'OCCUPATION_TYPE_Core staff',
 'OCCUPATION_TYPE_Drivers',
 'OCCUPATION_TYPE_HR staff',
 'OCCUPATION_TYPE_High skill tech staff',
 'OCCUPATION_TYPE_IT staff',
 'OCCUPATION_TYPE_Laborers',
 'OCCUPATION_TYPE_Low-skill Laborers',
 'OCCUPATION_TYPE_Managers',
 'OCCUPATION_TYPE_Medicine staff',
 'OCCUPATION_TYPE_Private service staff',
 'OCCUPATION_TYPE_Realty agents',
 'OCCUPATION_TYPE_Sales staff',
 'OCCUPATION_TYPE_Secretaries',
 'OCCUPATION_TYPE_Security staff',
 'OCCUPATION_TYPE_Unknown',
 'OCCUPATION_TYPE_Waiters/barmen staff'

 ]
numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
])

### Treinando para vários algoritmos

In [ ]:
  clf_forest = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=123))
  ])

  clf_logreg = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  clf_gb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GradientBoostingClassifier(random_state=123))
  ])

  clf_xgb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', XGBClassifier(random_state=123))
  ])

  clf_lgbm = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LGBMClassifier(random_state=123))
  ])

  clf_catboost = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', CatBoostClassifier(random_state=123, verbose=0))
  ])

  clf_ridge = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RidgeClassifier())
  ])

  clf_sgd = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SGDClassifier(random_state=123))
  ])

  clf_svc = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SVC(random_state=123))
  ])

  clf_knn = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', KNeighborsClassifier())
  ])

  clf_nb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GaussianNB())
  ])

  clf_mlp = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', MLPClassifier(random_state=123))
  ])

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    'SVC': clf_svc,
    'KNeighbors': clf_knn,
    'GaussianNB': clf_nb,
    'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Avaliando para várias métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)
    relatorio = {'accuracy': accuracy_score(y_test, y_pred), 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy                model
0   0.747866        Random Forest
1   0.739987  Logistic Regression
2   0.748851    Gradient Boosting
3   0.777851              XGBoost
4   0.766907             CatBoost
5   0.737470     Ridge Classifier
6   0.731451       SGD Classifier
7   0.750055                  SVC
8   0.705406           KNeighbors
9   0.698840           GaussianNB
10  0.746662        MLPClassifier


In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy    recall  precision       auc        f1                model
0   0.747866  0.664346   0.720034  0.736908  0.691070        Random Forest
1   0.739987  0.600670   0.738042  0.721708  0.662308  Logistic Regression
2   0.748851  0.682135   0.713592  0.740098  0.697509    Gradient Boosting
3   0.777851  0.729827   0.742460  0.771550  0.736089              XGBoost
4   0.766907  0.714617   0.730435  0.760047  0.722439             CatBoost
5   0.737470  0.584687   0.742147  0.717424  0.654074     Ridge Classifier
6   0.731451  0.603248   0.718894  0.714630  0.656013       SGD Classifier
7   0.750055  0.658675   0.726885  0.738065  0.691101                  SVC
8   0.705406  0.615880   0.665274  0.693660  0.639625           KNeighbors
9   0.698840  0.575664   0.668763  0.682679  0.618731           GaussianNB
10  0.746662  0.686775   0.707758  0.738805  0.697108        MLPClassifier


In [ ]:
#df_resultados.to_excel(r"Performance padrão dos algoritmos sem balanceamento.xlsx", index=False)

### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers = {}

for model_name, model in trained_models.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 50 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.159, mean = 0.421, max = 0.699
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.681, mean = 0.00394, max = 0.818
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

Para automatizar o processo de concatenar os resultados de desempenho de todos os modelos explicados com DALEX, irei iterar sobre os explicadores armazenados em um dicionário e concatenar seus resultados.

In [ ]:
performance_results = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name
    performance_results.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results = pd.concat(performance_results, ignore_index=True)

o método model_perfomance realiza o cálculo de medidas de desempenho do modelo


In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
model_fairness_results = {}

for model_name, explainer in explainers.items():
    fairness_result = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results[model_name] = fairness_result

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
fairness_checks = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result in model_fairness_results.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks = pd.DataFrame(fairness_checks)

# Exibir o DataFrame com os resultados
print(all_fairness_checks)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  0.982036  0.985353  0.998611  1.037234  1.012821
Métricas de viés detectadas para Random Forest: TPR=0.982036, ACC=0.985353, PPV=0.998611, FPR=1.037234, STP=1.012821
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  0.993355  0.991914  1.008152  1.012739  1.014535
Métricas de viés detectadas para 

Este código captura e processa a saída do método fairness_check() para cada modelo, extrai as métricas relevantes e armazena essas informações em um DataFrame para análise. Isso facilita a visualização e comparação dos resultados de diferentes modelos em termos de métricas de viés detectadas.

In [ ]:
all_fairness_checks.head()

,Modelo,Num Biases Detected,TPR,ACC,PPV,FPR,STP
0,Random Forest,None,0.982036,0.985353,0.998611,1.037234,1.012821
1,Logistic Regression,None,0.993355,0.991914,1.008152,1.012739,1.014535
2,Gradient Boosting,None,1.004405,0.994667,1.005618,1.035000,1.027363
3,XGBoost,None,0.974185,0.985915,1.001348,1.016129,1.000000
4,CatBoost,None,0.993017,0.994792,1.009615,1.005155,1.009662


# Segunda etapa do experimento - Balancear os dados para mitigar o viés

* dataset do compas
* balanceamento dos dados no treinamento
* várias técnicas de balanceamento dos dados
* vários algoritmos
* várias métricas de **fairness**

### Abordagem 1 - Embaralhar a coluna de sexo desbalanceada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET", 'ID'], axis =1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unknown,OCCUPATION_TYPE_Waiters/barmen staff
0,5008804,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5008805,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5008810,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
3,5008811,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
4,5008812,0,0,1,0,283500.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
X_train_abordagemUm = X_train.copy().reset_index(drop=True)

X_train_abordagemUm["CODE_GENDER"] = X_train["CODE_GENDER"].sample(frac=1).reset_index(drop=True)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_abordagemUm, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemUm = pd.DataFrame(resultados)
print(df_resultados_abordagemUm)


    accuracy    recall  precision       auc        f1                model
0   0.747975  0.667698   0.718646  0.737443  0.692236        Random Forest
1   0.740862  0.603506   0.738253  0.722841  0.664113  Logistic Regression
2   0.748194  0.681877   0.712554  0.739493  0.696878    Gradient Boosting
3   0.771722  0.721062   0.735859  0.765076  0.728385              XGBoost
4   0.767564  0.719515   0.729292  0.761260  0.724371             CatBoost
5   0.737032  0.583398   0.741967  0.716875  0.653197     Ridge Classifier
6   0.734734  0.593194   0.731173  0.716164  0.654996       SGD Classifier
7   0.748960  0.658675   0.724823  0.737115  0.690167                  SVC
8   0.692383  0.593968   0.650847  0.679471  0.621108           KNeighbors
9   0.698621  0.576695   0.667961  0.682624  0.618982           GaussianNB
10  0.744036  0.675947   0.707883  0.735102  0.691547        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 50 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.153, mean = 0.421, max = 0.722
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.694, mean = 0.00392, max = 0.821
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV      FPR  STP
0  1.030581  1.017591  1.001393  0.97449  1.0
Métricas de viés detectadas para Random Forest: TPR=1.030581, ACC=1.017591, PPV=1.001393, FPR=0.97449, STP=1.0
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.033898  1.009511  0.985255  1.045752  1.020468
Métricas de viés detectadas para Logistic Regressio

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [ ]:
fairness_result_balanceamento.plot()

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 2 - Duplicar os dados de treinamento com a classe oposta

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis =1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
df2 = X_train.copy()

df2["CODE_GENDER"] = df2["CODE_GENDER"].replace({1:0, 0: 1})
X_train2 = pd.concat([df2, X_train])
print(X_train2.shape, X_train.shape)
X_train2['CODE_GENDER'].value_counts()

(42642, 51) (21321, 51)


,count
CODE_GENDER,
1,21321
0,21321


In [ ]:
y_train2 = np.concatenate([y_train, y_train])

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados duplicados no treinamento com o sexo (classe) oposto
trained_models_duplicados = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train2, y_train2)
    trained_models_duplicados[model_name] = model
    print(f"Treinando {model_name} com dados de treinamento duplicados")

Treinando Random Forest com dados de treinamento duplicados
Treinando Logistic Regression com dados de treinamento duplicados
Treinando Gradient Boosting com dados de treinamento duplicados
Treinando XGBoost com dados de treinamento duplicados
Treinando CatBoost com dados de treinamento duplicados
Treinando Ridge Classifier com dados de treinamento duplicados
Treinando SGD Classifier com dados de treinamento duplicados
Treinando SVC com dados de treinamento duplicados
Treinando KNeighbors com dados de treinamento duplicados
Treinando GaussianNB com dados de treinamento duplicados
Treinando MLPClassifier com dados de treinamento duplicados


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_duplicados.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemDois = pd.DataFrame(resultados)
print(df_resultados_abordagemDois)


    accuracy    recall  precision       auc        f1                model
0   0.748194  0.671565   0.717236  0.738140  0.693649        Random Forest
1   0.739549  0.600412   0.737259  0.721294  0.661836  Logistic Regression
2   0.749398  0.683424   0.713978  0.740742  0.698367    Gradient Boosting
3   0.772817  0.716937   0.739824  0.765485  0.728201              XGBoost
4   0.771175  0.721836   0.734523  0.764702  0.728124             CatBoost
5   0.737360  0.584687   0.741904  0.717329  0.653979     Ridge Classifier
6   0.740096  0.642176   0.716216  0.727249  0.677178       SGD Classifier
7   0.750492  0.673885   0.720309  0.740441  0.696324                  SVC
8   0.708908  0.621294   0.669259  0.697413  0.644385           KNeighbors
9   0.699059  0.576953   0.668659  0.683038  0.619430           GaussianNB
10  0.744911  0.682650   0.706510  0.736742  0.694375        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_dois = {}

for model_name, model in trained_models_duplicados.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_dois[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.158, mean = 0.42, max = 0.727
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.683, mean = 0.00421, max = 0.815
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_dois = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_dois.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_dois.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_dois = pd.concat(performance_results_balanceamento_dois, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_dois = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_dois.items():
    fairness_result_balanceamento_dois = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_dois[model_name] = fairness_result_balanceamento_dois

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_dois = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_dois is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_dois.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_dois.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_dois = pd.DataFrame(fairness_checks_balanceamento_dois)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_dois)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.019608  1.016216  1.004196  0.950495  0.987531
Métricas de viés detectadas para Random Forest: TPR=1.019608, ACC=1.016216, PPV=1.004196, FPR=0.950495, STP=0.987531
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV      FPR       STP
0  1.005017  1.006793  0.993252  0.98125  0.982857
Métricas de viés detectadas para Lo

In [ ]:
# Gráfico de Barras: Comparação do Número de Métricas de Viés Detectadas por Modelo
fig = px.bar(
    all_fairness_checks_balanceamento_dois,
    x='Modelo',
    y='Num Biases Detected',
    title="Número de Métricas de Viés Detectadas por Modelo",
    color='Num Biases Detected',  # Colore as barras com base no valor
    color_continuous_scale='viridis',
    labels={"Num Biases Detected": "Número de Métricas de Viés", "Modelo": "Modelo"}
)

# Ajuste de layout para legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    title_x=0.5,  # Centraliza o título
    yaxis_title="Número de Métricas de Viés",
    xaxis_title="Modelo"
)

# Exibindo o gráfico
fig.show()

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_dois.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_dois.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier

Found NaN's or 0's for models: {'MLPClassifier'}
It is advisable to check 'metric_ratios'


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 3 - Subamostragem da Classe Majoritária

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Aplicar subamostragem da classe majoritária
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_subamostragem = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_res, y_train_res)
    trained_models_subamostragem[model_name] = model
    print(f"Treinado {model_name} com subamostragem")

Treinado Random Forest com subamostragem
Treinado Logistic Regression com subamostragem
Treinado Gradient Boosting com subamostragem
Treinado XGBoost com subamostragem
Treinado CatBoost com subamostragem
Treinado Ridge Classifier com subamostragem
Treinado SGD Classifier com subamostragem
Treinado SVC com subamostragem
Treinado KNeighbors com subamostragem
Treinado GaussianNB com subamostragem
Treinado MLPClassifier com subamostragem


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_subamostragem.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemTres = pd.DataFrame(resultados)
print(df_resultados_abordagemTres)


    accuracy    recall  precision       auc        f1                model
0   0.736923  0.788537   0.653721  0.744184  0.714828        Random Forest
1   0.753338  0.698508   0.707770  0.745624  0.703109  Logistic Regression
2   0.745568  0.760796   0.673228  0.747710  0.714338    Gradient Boosting
3   0.762968  0.788014   0.689489  0.766491  0.735467              XGBoost
4   0.761107  0.782779   0.688536  0.764156  0.732639             CatBoost
5   0.753885  0.679142   0.717247  0.743370  0.697674     Ridge Classifier
6   0.730794  0.689348   0.674175  0.724964  0.681677       SGD Classifier
7   0.752025  0.743784   0.688302  0.750865  0.714969                  SVC
8   0.682206  0.682544   0.606653  0.682254  0.642365           KNeighbors
9   0.669512  0.734363   0.583247  0.678635  0.650139           GaussianNB
10  0.744364  0.747187   0.675740  0.744761  0.709669        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_tres = {}

for model_name, model in trained_models_subamostragem.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_tres[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.176, mean = 0.485, max = 0.775
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.729, mean = -0.0665, max = 0.75
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_tres = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_tres.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_tres.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_tres = pd.concat(performance_results_balanceamento_tres, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_tres = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_tres.items():
    fairness_result_balanceamento_tres = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_tres[model_name] = fairness_result_balanceamento_tres

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_tres = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_tres is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_tres.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_tres.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_tres = pd.DataFrame(fairness_checks_balanceamento_tres)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_tres)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  0.998733  1.026243  1.023292  0.894737  0.950192
Métricas de viés detectadas para Random Forest: TPR=0.998733, ACC=1.026243, PPV=1.023292, FPR=0.894737, STP=0.950192
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.010086  1.018817  1.014265  0.913636  0.969121
Métricas de viés detectadas para 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_tres.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_tres.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 4 - Utilização da biblioteca SMOTE para criar exemplos sintéticos da classe minoritária - KMeans-SMOTE

In [ ]:
# Supomos que 'compas' seja seu DataFrame original
categorical_columns = [

 ]
non_categorical_columns =   df.drop(columns=categorical_columns)  # Selecionar as colunas restantes

# Inicializar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Aplicar o encoder nas colunas categóricas
encoded_data = encoder.fit_transform(df[categorical_columns]).toarray()  # Convertendo para array denso

# Criar um DataFrame com as colunas codificadas
encoded_df = pd.DataFrame(encoded_data,
                          columns=encoder.get_feature_names_out(categorical_columns),
                          index=df.index)  # Preserva os índices originais

# Concatenar o DataFrame codificado com as colunas não categóricas
df_encoded = pd.concat([non_categorical_columns, encoded_df], axis=1)

# Exibir o DataFrame final
df_encoded.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unknown,OCCUPATION_TYPE_Waiters/barmen staff
0,5008804,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5008805,1,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5008810,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
3,5008811,0,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
4,5008812,0,0,1,0,283500.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_encoded = df_encoded

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop(["TARGET"], axis=1),
                                                    df_encoded.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['CODE_GENDER'].astype(str) + "_" + y_train.astype(str)

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_group = smote.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_smote['CODE_GENDER'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_smote['CODE_GENDER'].value_counts())


1
1    16588
0    16588
Name: count, dtype: int64
CODE_GENDER
0    16588
1    16588
Name: count, dtype: int64


O SMOTE cria exemplos sintéticos da classe minoritária, aumentando o conjunto de dados de forma mais natural.

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_smote = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_smote, y_income)
    trained_models_smote[model_name] = model
    print(f"Treinando {model_name} com smote")

Treinando Random Forest com smote
Treinando Logistic Regression com smote
Treinando Gradient Boosting com smote
Treinando XGBoost com smote
Treinando CatBoost com smote
Treinando Ridge Classifier com smote
Treinando SGD Classifier com smote
Treinando SVC com smote
Treinando KNeighbors com smote
Treinando GaussianNB com smote
Treinando MLPClassifier com smote


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_smote.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemQuatro = pd.DataFrame(resultados)
print(df_resultados_abordagemQuatro)


    accuracy    recall  precision       auc        f1                model
0   0.738236  0.792201   0.654486  0.745828  0.716789        Random Forest
1   0.750164  0.674954   0.712431  0.739584  0.693186  Logistic Regression
2   0.748304  0.752421   0.679830  0.748883  0.714286    Gradient Boosting
3   0.770081  0.758440   0.710991  0.768443  0.733950              XGBoost
4   0.771941  0.762628   0.712295  0.770631  0.736603             CatBoost
5   0.750930  0.655588   0.722944  0.737517  0.687620     Ridge Classifier
6   0.732545  0.653232   0.690456  0.721388  0.671329       SGD Classifier
7   0.753338  0.720492   0.698908  0.748717  0.709536                  SVC
8   0.701138  0.633865   0.645178  0.691674  0.639472           KNeighbors
9   0.549683  0.794818   0.476916  0.584169  0.596133           GaussianNB
10  0.742613  0.712117   0.684873  0.738323  0.698229        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_quatro = {}

for model_name, model in trained_models_smote.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_quatro[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.166, mean = 0.476, max = 0.74
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.72, mean = -0.0576, max = 0.774
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data       

In [ ]:
performance_results_balanceamento_quatro = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_quatro.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_quatro.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_quatro = pd.concat(performance_results_balanceamento_quatro, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_quatro = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_quatro.items():
    fairness_result_balanceamento_quatro = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_quatro[model_name] = fairness_result_balanceamento_quatro

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_quatro = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_quatro is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_quatro.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])

        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_quatro.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_quatro = pd.DataFrame(fairness_checks_balanceamento_quatro)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_quatro)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV       FPR       STP
0  1.01145  1.022008  1.015432  0.927215  0.972868
Métricas de viés detectadas para Random Forest: TPR=1.01145, ACC=1.022008, PPV=1.015432, FPR=0.927215, STP=0.99784
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR      ACC       PPV       FPR       STP
0  1.069767  1.02439  1.001404  1.020725  1.041558
Métricas de viés detectadas para Logist

 graficos das metricas de fairness do dalex e avaliação para algunos modelos

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_quatro.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_quatro.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 5 - FairSmote (Implementação de um algoritmo que remove rótulos tendenciosos e rebalanceia as distribuições internas de forma que, com base no atributo sensível, os exemplos sejam iguais tanto nas classes positivas quanto negativas.)

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
from sklearn.neighbors import NearestNeighbors as NN

class smote(object):
  def __init__(self, pd_data, neighbor=5,r=2 ,up_to_num=[],auto=True):
    """
    :param pd_data: panda.DataFrame, the last column must be class label
    :param neighbor: num of nearst neighbors to select
    :param up_to_num: size of minorities to over-sampling
    :param up_to_max: if up_to_num is not supplied, all minority classes will
                      be over-sampled as much as majority class
    :return panda.DataFrame smoted data
    """
    self.set_data(pd_data)
    self.auto = auto
    self.neighbor = neighbor
    self.up_to_max = False
    self.up_to_num = up_to_num
    self.r = r
    self.label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #if up_to_num:
    #  label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #  if label_num - 1 != len(up_to_num):
    #    raise ValueError(
    #      "should set smoted size for " + str(label_num - 1) + " minorities")
    #  self.up_to_num = up_to_num
    #else:
    #  self.up_to_max = True

  def set_data(self, pd_data):
    if not pd_data.empty:# and isinstance(
        #pd_data.ix[:, pd_data.columns[-1]].values[0], str):
      self.data = pd_data
    else:
      raise ValueError(
        "The last column of pd_data should be string as class label")

  def get_majority_num(self):
    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    return majority_num

  def run(self):
    """
    run smote
    """

    def get_ngbr(data_no_label, knn):
      rand_sample_idx = random.randint(0, len(data_no_label) - 1)
      rand_sample = data_no_label[rand_sample_idx]
      distance, ngbr = knn.kneighbors(rand_sample.reshape(1, -1))
      # while True:
      rand_ngbr_idx = random.randint(0, len(ngbr))
      #   if distance[rand_ngbr_idx] == 0:
      #     continue  # avoid the sample itself, where distance ==0
      #   else:
      return data_no_label[rand_ngbr_idx], rand_sample

    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    for label, num in labelCont.items():
      if num < majority_num:
        to_add = majority_num - num
        last_column = self.data[self.data.columns[-1]]
        data_w_label = self.data.loc[last_column == label]
        data_no_label = data_w_label[self.data.columns[:-1]].values
        if len(data_no_label) < self.neighbor:
          num_neigh = len(data_no_label) # void # of neighbors >= sample size
        else:
          num_neigh = self.neighbor
        knn = NN(n_neighbors=num_neigh,p=self.r,algorithm='ball_tree').fit(data_no_label)
        if self.auto:
          to_add = to_add
        else:
          to_add = self.up_to_num
        for _ in range(to_add):
          rand_ngbr, sample = get_ngbr(data_no_label, knn)
          new_row = []
          for i, one in enumerate(rand_ngbr):
            gap = random.random()
            new_row.append(max(0, sample[i] + (
            sample[i] - one) * gap))  # here, feature vlaue should >=0
          new_row.append(label)
          total_data.append(new_row)
    return pd.DataFrame(total_data)

In [ ]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = smote(df)
    df = smt.run()
    df.columns = cols
    return df

In [ ]:
dataset_orig_train, dataset_orig_test = train_test_split(df_encoded, test_size=0.2, shuffle = True)

In [ ]:
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'TARGET'], dataset_orig_train['TARGET']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'TARGET'], dataset_orig_test['TARGET']

In [ ]:
train_df = X_train
train_df['TARGET'] = y_train

In [ ]:
train_df = apply_smote(train_df)

In [ ]:
y_train = train_df.TARGET
X_train = train_df.drop('TARGET', axis = 1)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemCinco = pd.DataFrame(resultados)
print(df_resultados_abordagemCinco)


    accuracy    recall  precision       auc        f1                model
0   0.746225  0.698142   0.701946  0.739892  0.700039        Random Forest
1   0.742942  0.644737   0.719965  0.730008  0.680278  Logistic Regression
2   0.753775  0.687693   0.719433  0.745073  0.703205    Gradient Boosting
3   0.762475  0.716718   0.721465  0.756449  0.719084              XGBoost
4   0.770683  0.724071   0.732290  0.764544  0.728157             CatBoost
5   0.736376  0.606811   0.726599  0.719312  0.661324     Ridge Classifier
6   0.733913  0.683050   0.687573  0.727215  0.685304       SGD Classifier
7   0.749015  0.664087   0.721918  0.737830  0.691796                  SVC
8   0.700427  0.616873   0.656237  0.689423  0.635947           KNeighbors
9   0.619665  0.345201   0.588003  0.583519  0.435016           GaussianNB
10  0.747702  0.717105   0.696879  0.743672  0.706847        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 6092 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.246, mean = 0.435, max = 0.684
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.675, mean = -0.0112, max = 0.753
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV       FPR       STP
0  1.00431  1.013532  1.038012  0.926087  0.992925
Métricas de viés detectadas para Random Forest: TPR=1.00431, ACC=1.013532, PPV=1.038012, FPR=0.926087, STP=0.992925
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR    STP
0  0.929204  0.989305  1.029745  0.880597  0.925
Métricas de viés detectadas para Logistic 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 6 - ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['CODE_GENDER'].astype(str) + "_" + y_train.astype(str)

# Aplicar ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_group = adasyn.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_adasyn['CODE_GENDER'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_adasyn['CODE_GENDER'].value_counts())


1
0    16730
1    15988
Name: count, dtype: int64
CODE_GENDER
1    16581
0    16137
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_adasyn, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.739987  0.779377   0.660164  0.745528  0.714834        Random Forest
1   0.750055  0.657420   0.720390  0.737023  0.687466  Logistic Regression
2   0.750383  0.740906   0.686803  0.749050  0.712829    Gradient Boosting
3   0.769424  0.750851   0.712972  0.766811  0.731421              XGBoost
4   0.767892  0.748757   0.711338  0.765200  0.729568             CatBoost
5   0.748413  0.634389   0.728803  0.732372  0.678327     Ridge Classifier
6   0.736485  0.649306   0.699070  0.724221  0.673270       SGD Classifier
7   0.754323  0.703742   0.707259  0.747207  0.705497                  SVC
8   0.703108  0.638053   0.647028  0.693956  0.642509           KNeighbors
9   0.559860  0.766815   0.483419  0.588975  0.592997           GaussianNB
10  0.751696  0.701910   0.703568  0.744692  0.702738        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.165, mean = 0.463, max = 0.736
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.711, mean = -0.0448, max = 0.796
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR      STP
1  1.005141  0.978523  0.978947  1.118705  1.05567
Métricas de viés detectadas para Random Forest: TPR=1.005141, ACC=0.978523, PPV=0.978947, FPR=1.118705, STP=1.05567
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR      ACC       PPV       FPR       STP
1  0.928678  0.97619  1.002778  0.956989  0.951031
Métricas de viés detectadas para Logis

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 7 - Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['CODE_GENDER'].astype(str) + "_" + y_train.astype(str)

# Agora balanceia os grupos combinados (ex: mulher_<=50K, homem_>50K, etc.)
ros = RandomOverSampler(random_state=42)
X_train_ros, y_group = ros.fit_resample(X_train, group)

# Separar novamente income e sex
#X_train_ros['sex'] = X_train_ros['sex']  # já está incluso
y_split = y_group.str.split('_', expand=True)
X_train_ros['CODE_GENDER'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)                # target original: income

# Verifica resultado
print(y_income.value_counts())
print(X_train_ros['CODE_GENDER'].value_counts())


1
1    16588
0    16588
Name: count, dtype: int64
CODE_GENDER
0    16588
1    16588
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_ros, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.730685  0.815755   0.639516  0.742653  0.716964        Random Forest
1   0.751149  0.699032   0.703821  0.743817  0.701418  Logistic Regression
2   0.746443  0.768647   0.672082  0.749567  0.717129    Gradient Boosting
3   0.772270  0.782518   0.705189  0.773711  0.741843              XGBoost
4   0.762202  0.774405   0.692974  0.763918  0.731430             CatBoost
5   0.754979  0.678618   0.719478  0.744237  0.698451     Ridge Classifier
6   0.742942  0.684376   0.695849  0.734702  0.690065       SGD Classifier
7   0.753557  0.752159   0.687724  0.753360  0.718500                  SVC
8   0.688882  0.674431   0.617098  0.686849  0.644492           KNeighbors
9   0.676625  0.701649   0.596308  0.680145  0.644704           GaussianNB
10  0.746662  0.739335   0.681708  0.745632  0.709353        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.153, mean = 0.483, max = 0.758
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.726, mean = -0.0649, max = 0.775
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.002454  0.987738  0.996875  1.061728  1.032197
Métricas de viés detectadas para Random Forest: TPR=1.002454, ACC=0.987738, PPV=0.996875, FPR=1.061728, STP=1.032197
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  0.970255  0.976222  0.984441  1.067633  1.012077
Métricas de viés detectadas para 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 8 -  NearMiss (remove exemplos da classe majoritária de forma informada, ou seja, não aleatoriamente, mas baseada na distância entre os exemplos da classe majoritária e os da classe minoritária.)

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['CODE_GENDER'].astype(str) + "_" + y_train.astype(str)

near = NearMiss()
X_train_near, y_group = near.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_near['CODE_GENDER'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_near['CODE_GENDER'].value_counts())


1
0    5948
1    5948
Name: count, dtype: int64
CODE_GENDER
0    5948
1    5948
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_near, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.715036  0.813923   0.621627  0.728948  0.704896        Random Forest
1   0.743160  0.705051   0.688298  0.737799  0.696574  Logistic Regression
2   0.730138  0.797174   0.643023  0.739569  0.711849    Gradient Boosting
3   0.716349  0.822560   0.621515  0.731291  0.708042              XGBoost
4   0.722915  0.819157   0.629652  0.736455  0.712011             CatBoost
5   0.745459  0.685946   0.699493  0.737086  0.692653     Ridge Classifier
6   0.717881  0.660822   0.663252  0.709854  0.662035       SGD Classifier
7   0.712300  0.758702   0.629396  0.718828  0.688027                  SVC
8   0.627380  0.674692   0.543882  0.634036  0.602266           KNeighbors
9   0.428321  0.975399   0.420797  0.505285  0.587948           GaussianNB
10  0.697527  0.780686   0.607659  0.709226  0.683391        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.186, mean = 0.509, max = 0.945
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.926, mean = -0.091, max = 0.775
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
    TPR       ACC       PPV       FPR       STP
1  0.96  1.009818  1.035772  0.912568  0.951439
Métricas de viés detectadas para Random Forest: TPR=0.96, ACC=1.009818, PPV=1.035772, FPR=0.912568, STP=0.951439
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  0.956643  0.982597  1.001453  0.991304  0.981439
Métricas de viés detectadas para Logistic Reg

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 9 - BorderlineSMOTE (Para superar a deficiência do SMOTE, ele identifica dois conjuntos de pontos — Ruído e Fronteira)

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['CODE_GENDER'].astype(str) + "_" + y_train.astype(str)

border = BorderlineSMOTE(random_state = 42, kind = 'borderline-2')
X_train_border, y_group = border.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_border['CODE_GENDER'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_border['CODE_GENDER'].value_counts())


1
1    16588
0    16588
Name: count, dtype: int64
CODE_GENDER
0    16588
1    16588
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_border, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.732436  0.812876   0.642266  0.743752  0.717570        Random Forest
1   0.751915  0.684638   0.711256  0.742450  0.697693  Logistic Regression
2   0.749617  0.755038   0.680906  0.750380  0.716059    Gradient Boosting
3   0.767564  0.759487   0.706598  0.766428  0.732089              XGBoost
4   0.765594  0.761842   0.702631  0.765066  0.731040             CatBoost
5   0.751806  0.666318   0.719412  0.739779  0.691848     Ridge Classifier
6   0.735281  0.679927   0.684765  0.727494  0.682337       SGD Classifier
7   0.749726  0.726511   0.690891  0.746461  0.708254                  SVC
8   0.699715  0.657681   0.636364  0.693802  0.646847           KNeighbors
9   0.541803  0.828841   0.472687  0.582184  0.602034           GaussianNB
10  0.749070  0.723633   0.690905  0.745491  0.706890        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.196, mean = 0.48, max = 0.726
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.711, mean = -0.0615, max = 0.775
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.007398  0.981004  0.982972  1.101587  1.049904
Métricas de viés detectadas para Random Forest: TPR=1.007398, ACC=0.981004, PPV=0.982972, FPR=1.101587, STP=1.049904
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC  PPV   FPR       STP
1  0.945481  0.978864  1.0  0.99  0.970443
Métricas de viés detectadas para Logistic Regressio

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 10 - Reweighting ( Reponderação sensível de classe) - Sample Weight


In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(["TARGET"], axis=1),
                                                    df.TARGET,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
sex_train = X_train['CODE_GENDER']  # ou como for sua variável

# Rebalancear o sexo no treino usando sample_weight
sex_counts = sex_train.value_counts(normalize=True)
sex_weights = {sex: 1.0 / freq for sex, freq in sex_counts.items()}

# Cria o vetor de pesos
sample_weight = sex_train.map(sex_weights).values

#### Treinando para vários algoritmos

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    #'SVC': clf_svc,
    #'KNeighbors': clf_knn,
    #'GaussianNB': clf_nb,
    #'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train, classifier__sample_weight=sample_weight)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


   accuracy    recall  precision       auc        f1                model
0  0.752462  0.679927   0.714325  0.742258  0.696702        Random Forest
1  0.744474  0.607171   0.735574  0.725158  0.665233  Logistic Regression
2  0.754760  0.687255   0.715142  0.745264  0.700921    Gradient Boosting
3  0.771722  0.725988   0.727511  0.765289  0.726749              XGBoost
4  0.769972  0.717351   0.728408  0.762569  0.722838             CatBoost
5  0.742942  0.590160   0.742265  0.721448  0.657530     Ridge Classifier
6  0.719413  0.634389   0.675021  0.707452  0.654074       SGD Classifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9138 rows 51 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9138 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e8b73950720> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.161, mean = 0.421, max = 0.708
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.675, mean = -0.00239, max = 0.81
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.CODE_GENDER

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.028486  1.022942  1.015559  0.931707  0.987531
Métricas de viés detectadas para Random Forest: TPR=1.028486, ACC=1.022942, PPV=1.015559, FPR=0.931707, STP=0.987531
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.023411  1.020436  1.010959  0.932927  0.985673
Métricas de viés detectadas para 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



# Resultados Finais

In [ ]:
all_performance_results.to_excel(r"Performance dos algoritmos sem balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento.to_excel(r"Performance dos algoritmos com 1º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_dois.to_excel(r"Performance dos algoritmos com 2º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_tres.to_excel(r"Performance dos algoritmos com 3º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_quatro.to_excel(r"Performance dos algoritmos com 4º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_fairness_checks.to_excel(r"Resultado da avaliação se os modelos são justos - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento.to_excel(r"Resultado da avaliação se os modelos são justos com 1º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_dois.to_excel(r"Resultado da avaliação se os modelos são justos com 2º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_tres.to_excel(r"Resultado da avaliação se os modelos são justos com 3º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_quatro.to_excel(r"Resultado da avaliação se os modelos são justos com 4º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)